In [1]:
%AddDeps com.databricks spark-csv_2.10 1.3.0 --transitive



:: loading settings :: url = jar:file:/opt/spark-1.5.1-bin-hadoop2.3/lib/spark-assembly-1.5.1-hadoop2.3.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
:: resolving dependencies :: com.ibm.spark#spark-kernel;working
	confs: [default]
	found com.databricks#spark-csv_2.10;1.3.0 in central
	found org.apache.commons#commons-csv;1.1 in central
	found com.univocity#univocity-parsers;1.5.1 in central
:: resolution report :: resolve 195ms :: artifacts dl 16ms
	:: modules in use:
	com.databricks#spark-csv_2.10;1.3.0 from central in [default]
	com.univocity#univocity-parsers;1.5.1 from central in [default]
	org.apache.commons#commons-csv;1.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   | 

In [2]:
import org.apache.spark.ml.clustering.{KMeans, KMeansModel}
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._

//Read raw data
val sqlContext = new SQLContext(sc)
val baseball = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load("baseball.csv")

//Set up features vectors
val toVector = udf[Vector, Double, Double, Double] { (a,b,c) => 
  Vectors.dense(a,b,c) 
}

val dataset = baseball.withColumn("features", toVector(baseball("PutOutRate"), 
                                                       baseball("AssistRate"),
                                                       baseball("ErrorRate") ))

// Trains a k-means model
val kmeans = new KMeans().setK(3).setFeaturesCol("features").setPredictionCol("prediction")
val model = kmeans.fit(dataset)
val transformed = model.transform(dataset)

//Add cluster label field
val toLabel = udf[String, Int] { (a) =>  "Cluster " + a}
val labeled = transformed.withColumn("cluster", toLabel(transformed("prediction")))


##Clustering by how a baseball player participates in outs likely indicates the position that they play
The top row of graphs shows the clusters, click on a cluster bar to see how the data for that cluster across other fields in the data in the bottom row of graphs.

In [4]:
%%brunel data('labeled') 
         x(cluster) y(#count:linear) bar interaction(select) color(cluster) legends(none) |
         x(AssistRate:linear) y(PutOutRate:linear) color(#count) bin(AssistRate, PutOutRate) style('symbol:rect')
            + x(AssistRate:linear) y(PutOutRate:linear) color(cluster) legends(none) |
         x(ErrorRate) y(#count) bin(ErrorRate) opacity(#selection) stack bar |
         x(BattingAvg) y(SlugRate) color(#selection)  
         
:: width=800, height=600